# Dask & OME-Zarr
Chunk-wise Volumetric Image Processing

In [1]:
import qim3d
from dask.distributed import Client
import dask.array as da
import zarr
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
from ome_zarr.io import ZarrLocation
from dask_image.ndfilters import uniform_filter

In [2]:
# Start Dask client with memory limit
client = Client(n_workers=1, memory_limit='1GB')

In [ ]:
# Load the OME-Zarr input file

location = ZarrLocation('Escargot.zarr')
reader = Reader(location)
nodes = list(reader())
image_node = nodes[0]
data = image_node.data[0]  # Assuming single scale level

data

In [ ]:
qim3d.viz.slicer(data, coarseness=4, image_size=6)

In [ ]:
# Apply operations on the volume
output = data
output = uniform_filter(output, size=(75,75,75))
output[200:400, 400:600, 400:600] = 255
output

In [ ]:
# Save the result to Output.zarr
output.to_zarr('Output.zarr', component='output', overwrite=True)
print('Saved thresholded data to Output.zarr')

In [ ]:
# Open the Zarr store directly
output_store = zarr.DirectoryStore('Output.zarr')
output_group = zarr.open(output_store, mode='r')

# Load the thresholded dataset as a Dask array
data_output = da.from_zarr(output_group['output'])
data_output

In [ ]:
qim3d.viz.slicer(data_output, coarseness=4, image_size=6)

In [ ]:
# Shutdown Dask client
# client.shutdown()